
## Brian's Week 3 Homework for Coursera Capstone part 3 ##

start off by doing everything from Brian's hw3 part2, but don't print as much out

scrape the Wikipedia page

In [1]:
#scrape table on wikipedia page
import requests
import lxml.html as lh
import pandas as pd

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')
[len(T) for T in tr_elements[:12]]

tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    #print (i,name)
    col.append((name,[]))

In [2]:
#len(tr_elements)

In [3]:

#print('parsing start')
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
   # print('parsing', j)
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        data
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
#print('done parsing table')        

In [4]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [5]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [6]:
#df.head()

Looks like we have the columns in the wrong order, so let's fix that now

In [7]:
#looks like we have the columns in the wrong order, so fix that

df = df[['Postcode', 'Borough', 'Neighbourhood\n']]#notice the \n in the third column - that took me a while to figure out
#df.head()

rename the columns to match the correct project name requirements

In [8]:
#rename columns to the project requirements
df.rename({'Neighbourhood\n': 'Neighborhood'}, axis='columns', inplace=True)
df.rename({'Postcode': 'PostalCode'}, axis='columns', inplace=True)
#df.head()

Clean up the dataframe

In [9]:
#clean up df and keep the old one
df_unclean=df
#df.shape
#take out the newlines in the data
df = df.replace('\n','', regex=True)
#lets look at the values we have and see that they make sense
#print(df['Neighborhood'].unique())
#print(df['Borough'].unique())
#print(df['PostalCode'].unique())

Get rid of rowns where Borough is not assigned

In [10]:
# get rid of rows where the Borough is not assigned
print(df.shape)
df=df[df['Borough']!='Not assigned']
#print(df.shape)
#print(df['Borough'].unique())

(288, 3)


If a neighborhood is not assigned, change it to match the borough name

In [11]:
#If a neighborhood is not assigned, change it to be the same as the borough
df.Neighborhood.value_counts()['Not assigned'] #how many do we have?
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] = df['Borough']
#df[0:10] #you can see that the Queen's Park neighborhood was assigned

In [12]:
all_postalcodes = df.PostalCode.unique()
#len(all_postalcodes)

combine multiple rows for a postalcode - combine the neighborhoods into commas separated lists

In [13]:
for postalcode in all_postalcodes:
    #print('\n\npostalcode=', postalcode, '\n')
    neighborhoods_to_combine = df[df['PostalCode']==postalcode].Neighborhood.unique()
    number_of_neighborhoods_to_combine = neighborhoods_to_combine.size
    neighborhoods_to_combine_csv_str = ', '.join(map(str, neighborhoods_to_combine))
                                      
    #print(number_of_neighborhoods_to_combine, ' - ', neighborhoods_to_combine_csv_str)
    #rows = df[df['PostalCode']==postalcode]
    #rows.Neighborhood=neighborhoods_to_combine_csv_str
    df.loc[df.PostalCode == postalcode, 'Neighborhood'] = neighborhoods_to_combine_csv_str
    
    #print(rows)
    #new_df.append(rows)


Combined the neighborhoods by postalcode, but didn't get rid of duplicate rows generated yet - let's get rid of duplicates now

In [14]:
print(df.shape)
df.drop_duplicates(inplace=True)
print(df.shape)
#df

(211, 3)
(103, 3)


fix the index

In [15]:
df.reset_index(inplace=True, drop=True)
#df

Last step for part 1 - print the datafram shape

In [16]:
df.shape

(103, 3)

done with Brian's hw3 part 1

## start the new stuff for Brian's hw3 part2 ##


Add latitude and longitude

In [17]:
#add column headers for latitude and longitude
header_list = ['PostalCode','Borough','Neighborhood','Latitude','Longitude']
df = df.reindex(columns = header_list)

The geocoder package never gave me results - it is too unreliable so I am commenting it

In [18]:
#I implemented the geocoder latitude and longitude,but it never returned results
'''
!conda install -c conda-forge geocoder -y
import geocoder # import geocoder

for postal_code in all_postalcodes:
    # initialize your variable to None
    lat_lng_coords = None
    print('\n',postal_code)
    # loop until you get the coordinates
    count=0
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng
      print('.',count,'.',postal_code,'.',g.latlng,'.')
      count=count+1
        
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print('\n',latitude,' ', longitude,'\n')
    df.loc[df.PostalCode == postalcode, 'Latitude','Longitude'] = latitude,longitude
'''

"\n!conda install -c conda-forge geocoder -y\nimport geocoder # import geocoder\n\nfor postal_code in all_postalcodes:\n    # initialize your variable to None\n    lat_lng_coords = None\n    print('\n',postal_code)\n    # loop until you get the coordinates\n    count=0\n    while(lat_lng_coords is None):\n      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n      lat_lng_coords = g.latlng\n      print('.',count,'.',postal_code,'.',g.latlng,'.')\n      count=count+1\n        \n    latitude = lat_lng_coords[0]\n    longitude = lat_lng_coords[1]\n    print('\n',latitude,' ', longitude,'\n')\n    df.loc[df.PostalCode == postalcode, 'Latitude','Longitude'] = latitude,longitude\n"

Get the latitudes and longitudes from the csv file since geocoder didn't work

In [19]:
lat_long_data = pd.read_csv('https://cocl.us/Geospatial_data')
lat_long_data.head()
for i in range(lat_long_data.Latitude.count()):
    lat = lat_long_data.iloc[i]['Latitude']
    lon = lat_long_data.iloc[i]['Longitude']
    pc = lat_long_data.iloc[i]['Postal Code']
    #print('\n',pc,'-',lat,'-',lon,'\n')
    df.loc[df.PostalCode == pc, 'Latitude'] = lat
    df.loc[df.PostalCode == pc, 'Longitude'] = lon

In [20]:
#does it look right?
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


show the dataframe with the latitude and the longitude

In [21]:
#df

## start Brian's hw3 part3 ##

In [22]:
#imports
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



I googled the initial latitude and longitude of Toronto

In [23]:
print('Libraries imported.')

# create map of Toronto using latitude and longitude values
toronto_lat = 43.6532
toronto_lon = -79.3832
map_toronto = folium.Map(location=[toronto_lat, toronto_lon], zoom_start=11)

neighborhoods = df
# add markers to map

#for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_toronto

Libraries imported.


NameError: name 'map_newyork' is not defined